In [1]:
def get_razd(sx_razd,v_):
    ch_razd = {}
    #     если это деловая часть
    for ind in sx_razd.index:

        if sx_razd.at[ind,'Из какой части пром']==0:
            v___ = v_ * sx_razd.at[ind,'Процент разделки']
    #         df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':fff.at[ind,'Часть'],'Объем':v___}))
            ch_razd[sx_razd.at[ind,'Часть']] =v___


    for ind in sx_razd.index:
    #     для пром
        if sx_razd.at[ind,'Из какой части пром']!=0:
    #         print(fff.at[ind,'Из какой части пром'])
    #         Процент вет брака
            pr_vetbr = sx_razd[sx_razd['Из какой части пром']==sx_razd.at[ind,'Из какой части пром']]['Процент разделки'].values
            v_pochasti = ch_razd[sx_razd.at[ind,'Из какой части пром']]
            it_prom = pr_vetbr * v_pochasti
            ch_razd[sx_razd.at[ind,'Часть']] = it_prom
            ch_razd[sx_razd.at[ind,'Из какой части пром']] = ch_razd[sx_razd.at[ind,'Из какой части пром']] - ch_razd[sx_razd.at[ind,'Часть']]
    
#     Выбираем минимальное из деловых частей (там где больше одного элемента)
    for ind in sx_razd.index:
        ch_razd[sx_razd.at[ind,'Часть']] = min(ch_razd[sx_razd.at[ind,'Часть']])
    
    for key___ in ch_razd.keys():
        ch_razd[key___] = ch_razd[key___].sum()
    return ch_razd

In [2]:
import pandas as pd
import datetime

In [3]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\analyze_zab\\unput\\'

In [4]:
ukpf =  pd.read_excel(directory+'УКПФ_заб.xlsx',sheet_name='Забой')
mpf =  pd.read_excel(directory+'МПФ_заб.xlsx',sheet_name='Забой')
chema_r = pd.read_excel(directory+'Схемы разделки.xlsx',sheet_name='Лист1')

In [5]:
# какие месяца по дням а какие по месяцам
params = {'days':[8],'month':[9,10,11,12]}

# Сколько с градации брать на разделку

dict_ ={'0,500-0,550':0, 
        '0,550-0,600':0, 
        '0,600-0,650':0,
        '0,650-0,700':0, 
        '0,700-0,750':0, 
        '0,750-0,800':0, 
        '0,800-0,850':0,
        '0,850-0,900':0, 
        '0,900-0,950':0, 
        '0,950-1,000':0, 
        '1,000-1,050':0,
        '1,050-1,100':0, 
        '1,100-1,150':0, 
        '1,150-1,200':0.2, 
        '1,200-1,250':0.7,
        '1,250-1,300':0.8, 
        '1,300-1,350':0, 
        '1,350-1,400':0, 
        '1,400-1,450':0,
        '1,450-1,500':0, 
        '1,500-1,550':0, 
        '1,550-1,600':0, 
        '1,600-1,650':0,
        '1,650-1,700':0, 
        '1,700-1,750':0, 
        '1,750-1,800':0, 
        '1,800-1,850':0,
        '1,850-1,900':0, 
        '1,900-1,950':0, 
        '1,950-2,000':0, 
        '2,000-2,050':0,
        '2,050-2,100':0, 
        '2,100-2,150':0, 
        '2,150-2,200':0, 
        '2,200-2,250':0,
        '2,250-2,300':0, 
        '2,300-2,350':0, 
        '2,350-2,400':0, 
        '2,400-2,450':0,
        '2,450-2,500':0, 
        '2,500-3,500':0}


# пропорция по схемам разделки
dict_sx = {1:0.5,2:0.3,3:0.2}

In [6]:
df_for_days = ukpf[[True  if x.month == params['days'][0] else False for x in ukpf['Дата забоя']]]
df_for_days_gr = df_for_days.groupby('Дата забоя').sum().reset_index()

In [7]:
df_cons = pd.DataFrame()
for date in df_for_days_gr['Дата забоя'].unique():
    
    df = df_for_days_gr[df_for_days_gr['Дата забоя']==date]
    
    for grad in dict_.keys():
        
        
        v_razd = df[grad].values * dict_[grad]
        v_cb = df[grad].values * (1- dict_[grad])
        
#         Часть с разделкой
        if v_razd>0:
            
#             print(v_razd)
            
#             Иду по пропорции разделки
            for el_sx in dict_sx.keys():
        
#                 Объем по схеме
                v_ = v_razd * dict_sx[el_sx]
#                 print(str(v_razd)+' '+ str(v_))
                sx_razd = chema_r[chema_r['Номер'] == el_sx]
                it_chema = get_razd(sx_razd,v_)
                for key_ in it_chema.keys():
                    df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':key_,'Объем':it_chema[key_]}))
#                     print(key_)
#                     break
#                 break
                
                
    
#                 print('общий объем'+ str(v_razd) + ' Объем на конкретную схему эту '+str(v_))
                
        
        
#         Часть ЦБ
        if v_cb>0:
            
            df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':'ЦБ','Объем':v_cb}))
            
        

In [8]:
df_cons = df_cons.groupby(['Дата забоя','Градация','Наименование части']).sum().reset_index()

df_cons.to_excel(directory+' test.xlsx')

In [9]:
df_for_days = ukpf[ukpf['Дата забоя'].dt.month.isin(params['month'])]
df_for_days['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_days['Дата забоя']]

C:\Users\8777~1\AppData\Local\Temp/ipykernel_15344/2610622463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_days['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_days['Дата забоя']]


In [10]:
df_for_days_gr = df_for_days.groupby('Дата забоя').sum().reset_index()

In [11]:
df_cons = pd.DataFrame()
for date in df_for_days_gr['Дата забоя'].unique():
    
    df = df_for_days_gr[df_for_days_gr['Дата забоя']==date]
    
    for grad in dict_.keys():
        
        
        v_razd = df[grad].values * dict_[grad]
        v_cb = df[grad].values * (1- dict_[grad])
        
#         Часть с разделкой
        if v_razd>0:
            
#             print(v_razd)
            
#             Иду по пропорции разделки
            for el_sx in dict_sx.keys():
        
#                 Объем по схеме
                v_ = v_razd * dict_sx[el_sx]
#                 print(str(v_razd)+' '+ str(v_))
                sx_razd = chema_r[chema_r['Номер'] == el_sx]
                it_chema = get_razd(sx_razd,v_)
                for key_ in it_chema.keys():
                    df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':key_,'Объем':it_chema[key_]}))
#                     print(key_)
#                     break
#                 break
                
                
    
#                 print('общий объем'+ str(v_razd) + ' Объем на конкретную схему эту '+str(v_))
                
        
        
#         Часть ЦБ
        if v_cb>0:
            
            df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':'ЦБ','Объем':v_cb}))

In [12]:
df_cons = df_cons.groupby(['Дата забоя','Градация','Наименование части']).sum().reset_index()
df_cons.to_excel(directory+' test1.xlsx')